# Perform a template fit of the for Data = a*QCD + b*VBF
## Using HistFactory in Root

In [ ]:
import ROOT
import os
import numpy as np

SF_sherpa = np.array([1.0,1.191947493608173, 1.0533849781514475, 0.9327344771000723, 0.829883020868467, 0.7448314828267345, 0.6511154483110894, 0.5930844551100937, 0.6242695391061465, 0.9092022336795254, 2.0082758450131486])
SF_sherpa_error =  np.array([0.0,0.024832248231732006, 0.02084994016932095, 0.025121873748490012, 0.03044693638877294, 0.03552656783662926, 0.04718166410747222, 0.08599269229591437, 0.17230228380603985, 0.5080358588861151, 2.167147164603685])
SF_mg =  np.array([1.0,0.7501716556638925, 0.6553409014338234, 0.5796489788329213, 0.5223969542591113, 0.48259319793002875, 0.4528394974758118, 0.461872031268506, 0.5186567944044667, 0.6895118832931821, 1.1046475273074001])
SF_mg_error =  np.array([0.0,0.017676865198085482, 0.01625302419125153, 0.02046071985681315, 0.024282779020102764, 0.028892515734035038, 0.04453963821747093, 0.09465078627174732, 0.19072927442291143, 0.45053134703576314, 1.0497978531225711])


def fillHistogramErrors(histogram,binErrors):
    for i in range(1,histogram.GetNbinsX()+1):
        histogram.SetBinError(i,binErrors[i-1])
        
def unpackHisto(histogram_file):
    bin_content_uncer =[]
    for i in range(1,histogram_file.GetNbinsX()+1):
        bin_content_uncer.append(histogram_file.GetBinError(i))
    return np.array(bin_content_uncer)
        
        
inputPath = "/Users/diegomac/Documents/HEP/VBF-Analysis/TauLep/ABCD/"

otherBGs = ['Wjets.root','VV.root','ttbar.root','singletop.root']
if "Tau" in inputPath:
    otherBGs += ['Higgs.root','Zjets.root']
samples = {"Signal":"Signal_Average.root","QCDjj":"Ztautau_MGRW.root","Data":"Data.root"}
qcd_shape_uncer = []

inputFile = ROOT.TFile.Open("histograms.root","RECREATE")
for i in samples:
    file = ROOT.TFile.Open(inputPath+samples[i])
    histogram = file.Get("mass_jj")
    x = np.array([   0.0,  250.0,  500.0,  750.0, 1000.0, 1250.0, 1500.0, 2000.0, 2500.0, 3000.0, 4000.0, 5000.0])
    
    if "Average" not in samples[i]:
        histogram = histogram.Rebin(len(x)-1,i,x)
    
    if i=="QCDjj":
        qcd_shape_uncer = unpackHisto(histogram)
        print('QCD Integral = ', histogram.Integral(1,-1))
        if "Sherpa" in samples["QCDjj"]:
            SF = SF_sherpa
            SF_error = SF_sherpa_error 
        elif "MG" in samples["QCDjj"]:
            SF = SF_mg
            SF_error = SF_mg_error 
        if "Average" not in samples[i]:
            qcd_shape_uncer = np.sqrt(qcd_shape_uncer**2+((SF_error/SF)**2)*qcd_shape_uncer**2) 
            fillHistogramErrors(histogram,qcd_shape_uncer)
        
    if i == "Data":
        for j in otherBGs:
            BGFile = ROOT.TFile.Open(inputPath+j)
            BGHistogram = BGFile.Get("mass_jj")
            BGHistogram = BGHistogram.Rebin(len(x)-1,j,x)
            
            histogram.Add(BGHistogram,-1.0)
            
    inputFile.WriteObject(histogram,i)
    file.Close()
    
inputFile.Close()

QCD Integral =  1059.4295114278793


In [93]:
os.system("root Fit.cpp | tail -n 2")

0

VBF = 174.687 +/- 53.3695
QCD = 1.13883 +/- 0.0675743


Info in <TCanvas::Print>: eps file ./results/example_UsingPy_channel1_meas_profileLR.eps has been created
Info in <TCanvas::Print>: eps file ./results/example_UsingPy_combined_meas_profileLR.eps has been created


## Using plain iminuit

In [26]:
import iminuit
import matplotlib.pyplot as plt
import numpy as np

from iminuit import cost, Minuit

def createHistogram(histogram):
    histo = []
    for i in histogram:
        histo.append([i,np.sqrt(i)])
    return np.array(histo)

yDataMeasurements = np.random.normal(91,4,1000)
yPredictionMeasurements = np.random.normal(91,4,100)

np.random.exponential()

data , x = np.histogram(yDataMeasurements,bins=200,range=(0,200))
prediction, x = np.histogram(yPredictionMeasurements,bins=200,range=(0,200))


dataHistogram = createHistogram(data)
predictionHistogram = createHistogram(prediction)

c = cost.Template(dataHistogram, x, np.array([predictionHistogram]))
m = Minuit(c,1000.0)
m.migrad()